In [1]:
# Check Phrasal paper to see how they split the corpus
# they only use en-fr from autodesk, not en-de

In [1]:
import codecs
import re
from collections import Counter, OrderedDict, defaultdict

In [16]:
# train_source_file = '/home/chris/projects/constrained_decoding/proto/autodesk_constrained_decoding_corpus/autodesk.train.en.bpe'
# train_target_file = '/home/chris/projects/constrained_decoding/proto/autodesk_constrained_decoding_corpus/autodesk.train.de.bpe'

source_tm_file = '/media/1tb_drive/parallel_data/wmt16_IT_domain/in_domain_train/indomain_tm/indomain.de-en.en.bpe'
target_tm_file = '/media/1tb_drive/parallel_data/wmt16_IT_domain/in_domain_train/indomain_tm/indomain.de-en.de.bpe'


In [3]:
gazetteer_file = '/media/1tb_drive/parallel_data/wmt16_IT_domain/in_domain_train/wiki_all/gazetteerDE.tsv'

In [4]:
from subword_nmt.apply_bpe import BPE

In [5]:
bpe_codes = open('/media/1tb_drive/parallel_data/en-de/chris_en-de_big_corpus/train/processed/all_text_both_EN_and_DE.79000.bpe.codes')

In [6]:
encoder = BPE(bpe_codes)

In [7]:
from subprocess import Popen, PIPE

# tokenization funcs
SOURCE_LANG = 'en'
TARGET_LANG = 'de'
# TARGET_LANG = 'fr'


tokenize_script = '/media/1tb_drive/parallel_data/en-de/chris_en-de_big_corpus/train/processed/tokenizer.perl'
source_tokenizer_cmd = [tokenize_script, '-l', SOURCE_LANG, '-q', '-', '-no-escape', '1']
target_tokenizer_cmd = [tokenize_script, '-l', TARGET_LANG, '-q', '-', '-no-escape', '1']

# NOTE: it's much slower to create a new tokenizer every time, see entity linking for persistent tokenizer
def tokenize(segment, src_trg):
    if src_trg == 'source':
        source_tokenizer = Popen(source_tokenizer_cmd, stdin=PIPE, stdout=PIPE)
        segment, _ = source_tokenizer.communicate(segment.encode('utf-8'))
    else:
        target_tokenizer = Popen(target_tokenizer_cmd, stdin=PIPE, stdout=PIPE)
        segment, _ = target_tokenizer.communicate(segment.encode('utf-8'))
    
    segment = segment.strip().decode('utf-8')
    return segment

In [8]:
def parse_gazetteer(filename):
    with codecs.open(filename, encoding='utf8') as inp:
        gazetteer_map = {}
        
        i = 0 
        for line in inp:
            deep, id_no, e, f = line.strip().split('\t')
            e = re.sub(u'"', u'', e)
            f = re.sub(u'"', u'', f)
            e = encoder.segment(tokenize(e, 'source'))
            f = encoder.segment(tokenize(f, 'target'))
            # TODO: tokenize first
            if i % 50 == 0:
                print(e)
                print(f)
            i += 1

            gazetteer_map[e] = f
    
    return gazetteer_map

In [9]:
# match English lowercase, map to foreign cased???? -- how much of a difference would this make?

In [10]:
gazet_map = parse_gazetteer(gazetteer_file)

english
title
Nur@@ sing inform@@ atics
Pfleg@@ ein@@ form@@ atik
Analy@@ st
Analy@@ st
BL@@ AST
BL@@ AS@@ T-@@ Algorithmus
Computer-@@ supported cooperative work
Computer Sup@@ ported Cooperative Work
DNA bar@@ coding
DNA-@@ Bar@@ coding
European Robo@@ tics Platform
EURO@@ P
Geo@@ information
Ge@@ od@@ aten
Ho@@ dg@@ kin – Hu@@ x@@ ley model
Ho@@ dg@@ k@@ in-@@ Hu@@ x@@ ley-@@ Modell
IT@@ C En@@ sch@@ ede
International Institute for Ge@@ o-@@ Information Science and Earth Observ@@ ation
Dag@@ stuhl
Leibni@@ z-@@ Zentrum f@@ ü@@ r Informatik
Compar@@ ison of web application frameworks
Liste von Web@@ frameworks
Mixed reality
Mixed reality
Open@@ Ajax Alliance
Open@@ Ajax Alliance
Protein sub@@ cellular localization prediction
Prot@@ ein@@ lok@@ alis@@ ations@@ vorher@@ sage
Ru@@ le@@ ML Symposium
Ru@@ le@@ ML Symposium
Spe@@ ech recognition
Spra@@ cher@@ kennung
Toyota Partner Robot
Toyota Partner Rob@@ ots
Web mining
Web Mining
Adel@@ e Gold@@ berg ( computer scientist )
Adel@@ e Gol

In [12]:
# LOAD dev source, get all constraints, write json file with those constraints

In [38]:
def get_segments_from_file(filename):
    with codecs.open(filename, encoding='utf8') as inp:
        for i, l in enumerate(inp):
            yield l.strip()

In [39]:
source_seg_iter = get_segments_from_file(source_tm_file)
target_seg_iter = get_segments_from_file(target_tm_file)

In [40]:
tm_pair_map = {k:v for k,v in zip(source_seg_iter, target_seg_iter)
                 if len(k) > 10 and len(k) < 50
                 and len(v) > 10 and len(v) < 50
                 and abs(len(k) - len(v)) < 15}
tm_rules = [src for src in tm_pair_map.keys()]

In [42]:
gazetteer_rules = [src for src,trg in gazet_map.items()]

In [43]:
# create terminology spotter
from semantic_annotator.spotting import MatchSpotter

In [44]:
# import the tokenized and bpe encoded dev lines
# prepped_dev_lines = codecs.open('/home/chris/projects/constrained_decoding/proto/autodesk_constrained_decoding_corpus/autodesk.dev.1000.en.bpe',
#                                  encoding='utf8').read().strip().split('\n')

# prepped_dev_lines = codecs.open('/home/chris/projects/constrained_decoding/proto/autodesk_constrained_decoding_corpus/en-fr/autodesk.dev.1000.en.bpe',
#                                  encoding='utf8').read().strip().split('\n')

prepped_dev_lines = codecs.open('/media/1tb_drive/parallel_data/wmt16_IT_domain/in_domain_test/refs/en-de/Batch3a_en.bpe',
                                 encoding='utf8').read().strip().split('\n')

In [46]:
# for k,v in tm_pair_map.items()[:100]:
#     print(u'key: {} val: {}'.format(k,v))

In [47]:
keys_by_len = sorted(tm_rules, key=lambda x: len(x))
print(keys_by_len[-100])
print(tm_pair_map[keys_by_len[-100]])


" Computer@@ ized maintenance management system "
" Computer@@ ized Maintenance Management System "


In [50]:
tm_spotter = MatchSpotter(rules=tm_rules)

# num constraints from TM
dev_term_spots = []
for l in prepped_dev_lines:
    spots = tm_spotter.get_spots(l)
    dev_term_spots.append(spots)

dev_term_constraints = []
for text, spots in zip(prepped_dev_lines, dev_term_spots):
    output_constraints = []
    if len(spots) > 0:
        for spot in spots:
            if spot[1] - spot[0] > 1:
                spotted_term = text[spot[0]:spot[1]]
#                 print(spotted_term)
                mapped_term = tm_pair_map[spotted_term]
                output_constraints.append(mapped_term.split())
    dev_term_constraints.append(output_constraints)
    
print(len([c for l in dev_term_constraints for c in l]))

270


In [53]:
# num constraints from Gazetteer
gazetteer_spotter = MatchSpotter(rules=gazetteer_rules)

# num constraints from TM
dev_term_spots = []
for l in prepped_dev_lines:
    spots = gazetteer_spotter.get_spots(l)
    dev_term_spots.append(spots)

dev_term_constraints = []
for text, spots in zip(prepped_dev_lines, dev_term_spots):
    output_constraints = []
    if len(spots) > 0:
        for spot in spots:
            if spot[1] - spot[0] > 1:
                spotted_term = text[spot[0]:spot[1]]
#                 print(spotted_term)
                mapped_term = gazet_map[spotted_term]
                output_constraints.append(mapped_term.split())
    dev_term_constraints.append(output_constraints)
    
print(len([c for l in dev_term_constraints for c in l]))

370


In [54]:
# combine both maps
combined_map = tm_pair_map.copy()
combined_map.update(gazet_map) # which returns None since it mutates combined_map

In [55]:
# num constraints from Gazetteer
combined_spotter = MatchSpotter(rules=[k for k,v in combined_map.items()])

# num constraints from TM
dev_term_spots = []
for l in prepped_dev_lines:
    spots = combined_spotter.get_spots(l)
    dev_term_spots.append(spots)

dev_term_constraints = []
for text, spots in zip(prepped_dev_lines, dev_term_spots):
    output_constraints = []
    if len(spots) > 0:
        for spot in spots:
            if spot[1] - spot[0] > 1:
                spotted_term = text[spot[0]:spot[1]]
#                 print(spotted_term)
                mapped_term = combined_map[spotted_term]
                output_constraints.append(mapped_term.split())
    dev_term_constraints.append(output_constraints)
    
print(len([c for l in dev_term_constraints for c in l]))

578


In [56]:
len(dev_term_constraints)

1000

In [57]:
import os
import json

# OUTPUT_DIR='/home/chris/projects/constrained_decoding/proto/autodesk_constrained_decoding_corpus'
# OUTPUT_DIR='/home/chris/projects/constrained_decoding/proto/autodesk_constrained_decoding_corpus/en-fr'
OUTPUT_DIR='/media/1tb_drive/parallel_data/wmt16_IT_domain/in_domain_test/refs/en-de/'

with codecs.open(os.path.join(OUTPUT_DIR, 'tm_and_gazet.test.constraints.json'), 'w', encoding='utf8') as out:
    out.write(json.dumps(dev_term_constraints, indent=2))